In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Import pandas and install pandasql
import pandas as pd
!pip install pandasql


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=f590e3f7bdd0833f5a86a89da6269409a102e9752528422fc3c774f1eba2f125
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [ ]:
# Load the zomato dataset and display its information
df = pd.read_csv('/content/drive/MyDrive/ZomData/zomato.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [ ]:
# Drop irrelevant columns
df2=df.drop(columns=['url','phone','rest_type','dish_liked','reviews_list','menu_item','listed_in(city)'],axis=1)

In [ ]:
# Display information about the DataFrame after dropping columns
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   address                      51717 non-null  object
 1   name                         51717 non-null  object
 2   online_order                 51717 non-null  object
 3   book_table                   51717 non-null  object
 4   rate                         43942 non-null  object
 5   votes                        51717 non-null  int64 
 6   location                     51696 non-null  object
 7   cuisines                     51672 non-null  object
 8   approx_cost(for two people)  51371 non-null  object
 9   listed_in(type)              51717 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.9+ MB


In [ ]:
# Rename columns for clarity
df2.rename({'listed_in(type)':'Restaurant_Type','rate':'Rating','approx_cost(for two people)':'Two_people_cost'},axis=1,inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   address          51717 non-null  object
 1   name             51717 non-null  object
 2   online_order     51717 non-null  object
 3   book_table       51717 non-null  object
 4   Rating           43942 non-null  object
 5   votes            51717 non-null  int64 
 6   location         51696 non-null  object
 7   cuisines         51672 non-null  object
 8   Two_people_cost  51371 non-null  object
 9   Restaurant_Type  51717 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.9+ MB


In [ ]:
# Check for null values in each column
print(df2.isnull().sum())

address               0
name                  0
online_order          0
book_table            0
Rating             7775
votes                 0
location             21
cuisines             45
Two_people_cost     346
Restaurant_Type       0
dtype: int64


In [ ]:
# Remove leading/trailing whitespace from the 'Rating' column
df2['Rating'] = df2['Rating'].str.strip()

In [ ]:
# Remove '/5' from the 'Rating' column
df2['Rating'] = df2['Rating'].str.replace('/5', '', regex=False)

In [ ]:
# Convert the 'Rating' column to numeric, coercing errors
df2['Rating'] = pd.to_numeric(df2['Rating'], errors='coerce')

In [ ]:
# Remove currency symbol and commas from 'Two_people_cost'
df2['Two_people_cost'] = df2['Two_people_cost'].str.replace('₹', '', regex=False)
df2['Two_people_cost'] = df2['Two_people_cost'].str.replace(',', '', regex=False)


In [ ]:
# Convert 'Two_people_cost' to numeric, coercing errors
df2['Two_people_cost'] = pd.to_numeric(df2['Two_people_cost'], errors='coerce')


In [ ]:
# Convert 'online_order' and 'book_table' to boolean
df2['online_order'] = df2['online_order'].str.strip().map({'Yes': True, 'No': False})
df2['book_table'] = df2['book_table'].str.strip().map({'Yes': True, 'No': False})


In [ ]:
# Display information about the DataFrame after type conversions
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          51717 non-null  object 
 1   name             51717 non-null  object 
 2   online_order     51717 non-null  bool   
 3   book_table       51717 non-null  bool   
 4   Rating           41665 non-null  float64
 5   votes            51717 non-null  int64  
 6   location         51696 non-null  object 
 7   cuisines         51672 non-null  object 
 8   Two_people_cost  51371 non-null  float64
 9   Restaurant_Type  51717 non-null  object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 3.3+ MB


In [ ]:
# Fill missing 'Rating' values with the median
df2['Rating'] = df2['Rating'].fillna(df2['Rating'].median())


In [ ]:
# Fill missing 'location' values with 'Unknown'
df2['location'] = df2['location'].fillna('Unknown')

In [ ]:
# Fill missing 'cuisines' values with 'Various'
df2['cuisines'] = df2['cuisines'].fillna('Various')

In [ ]:
# Fill missing 'Two_people_cost' values with the median
df2['Two_people_cost'] = df2['Two_people_cost'].fillna(df2['Two_people_cost'].median())

In [ ]:
# Display information about the DataFrame after filling missing values
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          51717 non-null  object 
 1   name             51717 non-null  object 
 2   online_order     51717 non-null  bool   
 3   book_table       51717 non-null  bool   
 4   Rating           51717 non-null  float64
 5   votes            51717 non-null  int64  
 6   location         51717 non-null  object 
 7   cuisines         51717 non-null  object 
 8   Two_people_cost  51717 non-null  float64
 9   Restaurant_Type  51717 non-null  object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 3.3+ MB


In [ ]:
# Verify that there are no more null values
print(df2.isnull().sum())

address            0
name               0
online_order       0
book_table         0
Rating             0
votes              0
location           0
cuisines           0
Two_people_cost    0
Restaurant_Type    0
dtype: int64


In [ ]:
# Display the shape of the DataFrame
df2.shape

(51717, 10)

In [ ]:
# Remove duplicate rows
df3=df2.drop_duplicates()

In [ ]:
# Display information about the DataFrame after dropping duplicates
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31924 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          31924 non-null  object 
 1   name             31924 non-null  object 
 2   online_order     31924 non-null  bool   
 3   book_table       31924 non-null  bool   
 4   Rating           31924 non-null  float64
 5   votes            31924 non-null  int64  
 6   location         31924 non-null  object 
 7   cuisines         31924 non-null  object 
 8   Two_people_cost  31924 non-null  float64
 9   Restaurant_Type  31924 non-null  object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 2.3+ MB


In [ ]:
# Calculate cost per person by dividing by 2
df3.loc[:, 'Two_people_cost'] = df3['Two_people_cost'] / 2

In [ ]:
# Rename the 'Two_people_cost' column to 'cost_per_person'
df3=df3.rename(columns={'Two_people_cost':'cost_per_person'})

In [ ]:
# Save the cleaned DataFrame to a CSV file
df3.to_csv("zomato_data_analysis.csv")

In [ ]:
# Display information about the final DataFrame
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31924 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          31924 non-null  object 
 1   name             31924 non-null  object 
 2   online_order     31924 non-null  bool   
 3   book_table       31924 non-null  bool   
 4   Rating           31924 non-null  float64
 5   votes            31924 non-null  int64  
 6   location         31924 non-null  object 
 7   cuisines         31924 non-null  object 
 8   cost_per_person  31924 non-null  float64
 9   Restaurant_Type  31924 non-null  object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 2.3+ MB


In [ ]:
# Apply title casing to key categorical text columns
df3['name'] = df3['name'].str.title()
df3['location'] = df3['location'].str.title()
df3['cuisines'] = df3['cuisines'].str.title()
df3['Restaurant_Type'] = df3['Restaurant_Type'].str.title()


In [ ]:
# Display the head of selected columns after title casing
df3[['name', 'location', 'cuisines', 'Restaurant_Type', 'online_order', 'book_table']].head()

,name,location,cuisines,Restaurant_Type,online_order,book_table
0,Jalsa,Banashankari,"North Indian, Mughlai, Chinese",Buffet,True,True
1,Spice Elephant,Banashankari,"Chinese, North Indian, Thai",Buffet,True,False
2,San Churro Cafe,Banashankari,"Cafe, Mexican, Italian",Buffet,True,False
3,Addhuri Udupi Bhojana,Banashankari,"South Indian, North Indian",Buffet,False,False
4,Grand Village,Basavanagudi,"North Indian, Rajasthani",Buffet,False,False


In [ ]:
# Save the cleaned DataFrame to Google Drive
df3.to_csv('/content/drive/MyDrive/zomato_cleaned_data.csv', index=False)


In [ ]:
# Convert column names to lowercase and display info
df3.columns = df3.columns.str.lower()
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31924 entries, 0 to 51716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   address          31924 non-null  object 
 1   name             31924 non-null  object 
 2   online_order     31924 non-null  bool   
 3   book_table       31924 non-null  bool   
 4   rating           31924 non-null  float64
 5   votes            31924 non-null  int64  
 6   location         31924 non-null  object 
 7   cuisines         31924 non-null  object 
 8   two_people_cost  31924 non-null  float64
 9   restaurant_type  31924 non-null  object 
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 2.3+ MB


In [ ]:
# Save the DataFrame with lowercase column names to Google Drive
df3.to_csv('/content/drive/MyDrive/zomato_cleaned_data.csv', index=False)